# Curve Fitting with `lmfit`
This notebook demonstrates how to use the `lmfit` library for fitting a the model parameters of a synthetic dataset using different optimization methods. We will generate some noisy data, define a model function, and then fit the data using various methods.


In [ ]:
import numpy as np
from lmfit import Minimizer, create_params, report_fit
import matplotlib.pyplot as plt

## Generating Synthetic Data
We will create a dataset based on a damped sinusoidal function with added noise on the form 

$$
y= A \text{sin}(\omega x + \phi ) \text{e}^{-\gamma x^2 }.
$$

Setting a random seed ensures the reproducibility of the results.


In [ ]:
np.random.seed(2021)  # Set random seed for reproducibility

amplitude = 5.0
omega = 2.0
phi = -0.1
gamma = 0.025

x = np.linspace(0, 15, 301)
data = amplitude * np.sin(omega * x + phi) * np.exp(-x * x * gamma) + np.random.normal(
    size=x.size, scale=0.2
)

plt.figure(figsize=(10, 6))
plt.scatter(x, data, label="Data", color="red", alpha=0.6)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

## Defining the Model and Residual Function
The residual function calculates the difference between the data and the model prediction. This function is used by the optimization algorithm during the fitting process.


In [ ]:
def residual(parameters, x, data):
    model = (
        parameters["amplitude"]
        * np.sin(x * parameters["omega"] + parameters["phi"])
        * np.exp(-x * x * parameters["gamma"])
    )
    return model - data

## Initial Parameter Setup
We need to provide initial guesses for the fitting parameters. These guesses will be changed by the chosen minimization algorithm and hopefully optimized.


In [ ]:
parameters_initial = create_params(
    amplitude=dict(value=10, min=0),
    gamma=0.1,
    omega=3.0,
    phi=dict(value=0.2, min=-np.pi / 2.0, max=np.pi / 2),
)

initial_model = (
    parameters_initial["amplitude"].value
    * np.sin(x * parameters_initial["omega"].value + parameters_initial["phi"].value)
    * np.exp(-x * x * parameters_initial["gamma"].value)
)

plt.figure(figsize=(10, 6))
plt.scatter(x, data, label="Data", color="red", alpha=0.6)
plt.plot(x, initial_model, label="Initial guess", linestyle="--", color="green")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

## Performing the Minimization
We will use the `lmfit.Minimizer` to minimize the residuals using the specified method.


In [ ]:
method = "leastsq"  # You can play with different methods e.g.: "nelder"
minimizer = Minimizer(residual, parameters_initial, fcn_args=(x, data))
result = minimizer.minimize(method=method)

final_model = data + result.residual

report_fit(result)

## Visualizing the Fit Results
Now we compare the noisy data, the initial model guess, and the final fitted model.


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x, data, label="Data", color="red", alpha=0.6)
plt.plot(x, initial_model, label="Initial guess", linestyle="--", color="green")
plt.plot(x, final_model, label="Fit", color="blue")
plt.title(f"Optimization using {method} method")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()